<a href="https://colab.research.google.com/github/manasarthak/Emotion-classification-using-physiological-signal/blob/main/Loading_DEAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from IPython.utils import io
import numpy as np
import collections

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.utils import shuffle

import scipy.io
from scipy import signal,integrate
import matplotlib.pyplot as plt

import keras
from keras.models import Model
from keras.layers import Input,Dense,LSTM,Dropout
from keras import optimizers

import mne 
import math

import pickle

In [5]:
def convert(subject_file):
    pth=pickle.load(open(subject_file,'rb'),encoding='latin1');
    data=pth['data'][:,0:32,13*128:-10*128]
  #excluding the first 3 seconds of baseline signal data and 10 seconds at the satrt and finish to counter residual emotions and exhaustion
    labels=pth['labels'][:,0:2]#only the valence and arousal dimensions 
    valence_labels,valence_data=[],[];
    arousal_labels,arousal_data=[],[];
    for i, label in enumerate(labels):
        valence,arousal=label[0],label[1]
        if(valence>=5):
            valence_labels.append(1);
            valence_data.append(data[i]);
        if(valence<5):
            valence_labels.append(0);
            valence_data.append(data[i]);
        if arousal >=5:
            arousal_labels.append(1);
            arousal_data.append(data[i]);
        if arousal <5:
            arousal_labels.append(0);
            arousal_data.append(data[i]);
            
    print("valence:",len(valence_labels),"arousal: ",len(arousal_labels));
    return valence_labels,valence_data,arousal_labels,arousal_data;

In [8]:
def standardize(a,multiple):
    std=np.std(a)
    mean=np.mean(a)
    a=(a-mean)/std
    return multiple*a

In [9]:
def convertAll():
    valence_labels_all,valence_data_all=[],[];
    arousal_labels_all,arousal_data_all=[],[];
    
    for i in range(32):
        if(i<9):
            file="0"+str(i+1);
        else:
            file=str(i+1);
        filename='/content/drive/MyDrive/DEAP/s'+file+'.dat';
        print(filename);
        valence_labels,valence_data,arousal_labels,arousal_data=convert(filename);
        valence_labels_all+=valence_labels
        for d in valence_data:
            d=standardize(d,1)
            valence_data_all.append(d)
        arousal_labels_all+=arousal_labels
        for d in arousal_data:
            d=standardize(d,1)
            arousal_data_all.append(d)
    valence_labels_all=np.array(valence_labels_all)
    valence_data_all=np.array(valence_data_all)
    arousal_labels_all=np.array(arousal_labels_all)
    arousal_data_all=np.array(arousal_data_all)
    
    print("Valence trial data for all subject: ", valence_labels_all.shape,valence_data_all.shape)
    print("Arousal trial data for all subject: ", arousal_labels_all.shape,arousal_data_all.shape)
    
    # save numpy array of total data to files
    np.save('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_labels.npy',valence_labels_all)
    np.save('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_data.npy', valence_data_all)
    np.save('/content/drive/MyDrive/DEAP/arousal/' + 'all_arousal_labels.npy',arousal_labels_all)
    np.save('/content/drive/MyDrive/DEAP/arousal/' + 'all_arousal_data.npy', arousal_data_all)

In [11]:
convertAll();

/content/drive/MyDrive/DEAP/s01.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s02.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s03.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s04.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s05.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s06.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s07.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s08.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s09.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s10.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s11.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s12.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s13.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s14.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s15.dat
valence: 40 arousal:  40
/content/drive/MyDrive/DEAP/s16.dat
valence: 40 arousal:  40
/content/drive/MyDrive/D